<a href="https://colab.research.google.com/github/Praveen76/Introduction-to-Huggingface/blob/main/Fine_Tunning_BERT_Model_on_GLUE_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Huggingface: Fine Tunning BERT using Built in Data**

### Objectives:

At the end of the experiment you will be able to understand  :

* How datasets are represented in HF
* Trainer and trainingArguments objects
* Computing Metrics
* Saving and loading the trained model

In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━ 193.0/410.6 MB 163.6 MB/s eta 0:00:02

Accelerate is a library that enables the same PyTorch code to be run across any distributed configuration by adding just four lines of code, making training and inference at scale made simple, efficient and adaptable.

In [ ]:
!pip install transformers

Hugging Face Datasets Library


In [ ]:
!pip install datasets

### **Loading the dataset**
Built in data sets are included with the data sets package. Using a single line of code, we can load in the data set  which  are well formatted  and processed for us to use.

In [ ]:
import numpy as np
from datasets import load_dataset
#rd1=load_dataset("amazon_polarity") # dataset for sentiment analysis
rd2=load_dataset("glue","sst2")     # Part of GLUE Benchmark

Calling the load data set function, returns a data set dict object, which is like a dictionary that may contain multiple data sets like -  a train dataset, validation dataset and test dataset.

In [ ]:
rd2

Getting different data and individual entries

In [ ]:
rd2['train']

In [ ]:
print(rd2['train'], '\n')
type(rd2['train'])

In [ ]:
rd2['train'].data

In [ ]:
rd2['train'][0]

In [ ]:
rd2['train'][10:13]

In [ ]:
rd2['train'].features

### **Tokenization with DataSets**
Recall that previously, we mentioned the need for truncation, padding and converting the tokenized data into PyTorch Tensors. Here, all of these steps will be handled  behind the scenes and no need to do anything. We are going to do is to partially apply the tokenizer to the data set dict object.



In [ ]:
from transformers import AutoTokenizer

In [ ]:
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
tokenized_sentence = tokenizer(rd2['train'][0:3]['sentence'])
from pprint import pprint
pprint(tokenized_sentence)

By calling the map method and passing in a tokenized function, the data set library automatically knows it should apply the same function to every data set i.e.train, validation and test.

 Note that in this example we apply only truncation but not padding or conversion into Pytorch Tensors. This will be handled behind the scenes by the trainer object we create later.

In [ ]:
def tokenize_fn(batch):
  return tokenizer(batch['sentence'],truncation=True)

In [ ]:
tokenized_datasets=rd2.map(tokenize_fn,batched=True)

### **Loading Pre-Trained Model**

In [ ]:
from transformers import AutoModelForSequenceClassification
model =AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=2)

In [ ]:
type(model)

In [ ]:
model

In [ ]:
!pip install torchinfo

In [ ]:
from torchinfo import summary
summary(model)

#### **Adding custom classifier layter and Freezing/Un-Freezing different layers**

In [ ]:
for name, param in model.named_parameters():
  print(name)

In [ ]:
# Freezing
for name, param in model.named_parameters():
     if name.startswith("distilbert"): # choose whatever you like here
        param.requires_grad = False

In [ ]:
# Displaying of layers gradient
for name, param in model.named_parameters():
     print(name, param.requires_grad)

#### **Modifying last classifier layers**

In [ ]:
import torch

In [ ]:
model.pre_classifier

In [ ]:
print(model.pre_classifier)
model.pre_classifier = torch.nn.Linear(768,500)
print(model.pre_classifier)

In [ ]:
print(model.classifier)
model.classifier = torch.nn.Linear(500,3)
print(model.classifier)

In [ ]:
model

#### **Adding extra layer on top**

In [ ]:
import copy

In [ ]:
cp_model = copy.deepcopy(model)

In [ ]:
cp_model.classifier = torch.nn.Sequential(
    (torch. nn.Linear(768, 192)),
    torch.nn.Dropout(0.1),
    (torch.nn.Linear(192, 2)))

In [ ]:
cp_model

In [ ]:
params_before=[]
for  name,p in model.named_parameters():
  params_before.append(p.detach().cpu().numpy())

In [ ]:
params_before

### **Metrics**

In HuggingFace, there are predefined metrics for the built in data sets and we are going to use that but we can define custom metrics as well.

Specifically, there is a function called the load metric, also part of the data sets library. The arguments to this happens to be the same as the arguments into load data set for this example.

More generally, however, you can pass in an argument to specify a specific metric like the Blue score or a python file where you've defined your own computations.

In [ ]:
from datasets import load_metric
metric =load_metric("glue","sst2")

But,  we cannot pass this metric into the trainer directly. we need to define another function to do some extra work.

Firstly, the metric we get back from `load_metric` is an object which has a method called `compute`, which will return a dictionary containing the computed metrics for a given task. This compute method it takes in to erase the predictions and the targets called a references.

This will give us back a dictionary with the value of the metric or metrics with their names as keys.

In [ ]:
metric.compute(predictions=[1,0,1], references=[1,0,0])

In [ ]:
def compute_metrics(logits_and_labels):
  logits, labels = logits_and_labels
  predictions =np.argmax(logits,axis=-1)
  return metric.compute(predictions=predictions,references=labels)

### **Training argument class**


The training arguments class is like a training configuration. It allows to specify things like  - `where to save the outputs of the training process, how often to compute metrics, how many epochs to train for the learning rate, and many more`. There are tons of arguments into the constructor for this class. So check out the documentation.

Note that the HuggingFace trainer uses an optimizer called **`Adam W`** by default, which is a slight variation of Adam.This is essentially built in and it's good enough for our purposes.If you want to use a different optimizer, you can always write your own custom training loop in plain Pytorch syntax as well.

In [ ]:
from transformers import TrainingArguments

In [ ]:
training_args = TrainingArguments(
    'sample_trainer', #  directory where the trained model will be saved,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    num_train_epochs=1

)

### **Trainer**

Trainer object is what we use to run the training process. The arguments to the constructor are quite simple. We pass in the model, the trainer arguments, the training data set, the validation data set, the tokenizer and the metrics.

Now, just call `trainer` and apply `train` method to train.



In [ ]:
from transformers import Trainer

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["validation"],
    tokenizer =tokenizer,
    compute_metrics=compute_metrics

)

In [ ]:
trainer.train()
 # ~ 2 mint for tunning just last classifier layers
 # ~ 8 mint for tunning whole architecture

#### **Saving the model**

In [ ]:
trainer.save_model('saved_model')

In [ ]:
!ls

#### **Loading the saved model**

In [ ]:
from transformers import pipeline

In [ ]:
my_model = pipeline('text-classification',model='saved_model',device=0)# ignore_mismatched_sizes=True)

In [ ]:
my_model('this is not a good book to read')

#### **Checking whether all parameters have updated or not**

In [ ]:
params_after=[]
for  name,p in model.named_parameters():
  params_after.append(p.detach().cpu().numpy())

In [ ]:
for p1,p2 in zip(params_before, params_after):
  print(np.sum(np.abs(p1-p2)))